# Evaluation of the Simplex Algorithm
EN.605.621.31 Foundations of Algorithms <br>
Final Project - Part 1
Due 10/14/2019

## Linear Programming


Linear programming is an method used to solve linear minimization and maximation problems given a set of constraints and resource limitations. Although this approach is commonly used in mathematics, it is also broadly applicable to other industry verticals including business and economics as well as engineering and manufacturing. Consider a manufacturer who creates school supplies. They want to determine how many of notebook, pencils, and binders to produce. However, they have a limited set of raw materials and labor hours. How can they determine the optimal mix of products to minimize costs and maximize profit? 

This is a very simplified example of a linear programming problem. Although these problems may vary in complexity, a standard linear programing problem consists of three components: <br>
- Variables: *x =( x1,x2,...,xd)^T* <br>
- Objective Function: *c·x* <br>
- Constraints: *Ax ≤ b* (where A is a n×m matrix) <br>

To simplify and standardize linear programs, they are often expressed in a standard form where all constraints are inequalities. For example:
- Maximize: *(c^T)x*
- Subject to:
    - *Ax <= b*
    - *x >= 0*
    
Using this format, a linear program can then be represented in the form of a tuple *(A, b, c)*.

*[TBD -- add more to this section, probably]*

There are multiple algorithms designed to solve linear programming problems including the ellipsoid algorithm and the interior-point methods. Although these algorithms both run in polynomial time, they can be complex and run slowly in practice. The approach analyzed in this report is the Simplex method. While the worst-case runtime complexity is exponential, it often runs quickly in practice. 


## The Simplex Algorithm
The Simplex Algorithm, also referred to as the “Simplex Method” was invented by George Dantzig in 1947 (Carreira-Perpiñán). The Simplex Algorithm can be leveraged to solve linear programing problems and to identify the optimal solution from the set of all possible solutions. The Simplex Method follows an iterative approach. In each iteration, the problem is rewritten in an equivalent form. After a few iterations, the solution to the problem should be simpler to obtain. Essentially, the challenging linear program is iteratively rewritten until the optimal solution is easy to identify. 


*[TBD -- add more to this section, probably]*


## Algorithm Code

In [7]:
import numpy as np
from scipy.optimize import linprog # 'revised-simplex' requires scipy 1.3 or above
import random


def pivot(N, B, A, b, c, v, l, e):  # page 869
    # Compute the coefficients of the equation for new basic variable x_e.
    (m, n) = A.shape; A_hat = np.empty((m, n)); b_hat = np.empty(m); c_hat = np.empty(n)
    b_hat[l] = b[l]/A[l][e]
    for j in range(n):
        if j != e:
            A_hat[l][j] = A[l][j]/A[l][e]
    A_hat[l][e] = 1/A[l][e]
    # Compute the coefficients of the remaining contraints.
    for i in range(m):
        if i != l:
            b_hat[i] = b[i] - A[i][e]*b_hat[l]
            for j in range(n):
                if j != e:
                    A_hat[i][j] = A[i][j] - A[i][e]*A_hat[l][j]
            A_hat[i][e] = -A[i][e]*A_hat[l][e]
    # Compute the objective function.
    v += c[e]*b_hat[l]
    for j in range(n):
        if j != e:
            c_hat[j] = c[j] - c[e]*A_hat[l][j]
    c_hat[e] = -c[e]*A_hat[l][e]
    # Compute new sets of basic and nonbasic variables.
    B[l], N[e] = N[e], B[l]
    return (N, B, A_hat, b_hat, c_hat, v)


def simplex(A, b, c):  # page 871
    (N, B, A, b, c, v) = initalize_simplex(A, b, c)
    (m, n) = A.shape; delta = np.empty(m); x_bar = np.zeros(n)
    while np.any(c > 0):
        e = np.where(c > 0)[0][0]
        for i in range(m):
            if A[i][e] > 0:
                delta[i] = b[i]/A[i][e]
            else:
                delta[i] = np.inf
        l = np.where(delta == np.amin(delta))[0][0]
        if delta[l] == np.inf:
            return "unbounded"
        else:
            (N, B, A, b, c, v) = pivot(N, B, A, b, c, v, l, e)
    for i in range(m):
        if B[i] < n:
            x_bar[B[i]] = b[i]
    return x_bar, v


def initalize_simplex(A, b, c):  # page 887
    if np.all(b >= 0):  # is the initial basic solution feasible?
        m, n = A.shape; return (np.arange(n), np.arange(m)+n, A, b, c, 0)
    else:
        raise Exception('the initial basic solution is infeasible')

## Theoretical Analysis


[TBD]
1. Theoretical details about the algorithm and theoretical complexity
2. Details about the algorithm of choice: 
    2.1 Why did we study this algorithm? 
    2.2 What pieces does it have? 
        - Data structures 
        - algorithm strategies


## Emperical Analysis
Empirical analysis of the Simplex Algorithm will be performed by running the code with sample input data, assessing the performance of the algorithm, and comparing the results to the theoretical analysis. 

This input data will consist of both consistent test data and randomly  generated values for the following variables:
- A = *m x n* matrix<br> 
- b = *m*-vector <br>
- c = *n*-vector <br><br>

### Consistent Test Data 
The following data will be used to perfrorm the emperical analysis of the algorithm runtime.


##### Input 1: 2x2 matrix
A = [[1, 5],[2, 1]] <br> 
b = [7, 4] <br>
c = [2, 3] <br>
Optimality: 6.222 <br>

##### Input 2: 3x3 matrix
A = [[1, 1, -1],[-1, -1, 1],[1, -2, 2]] <br> 
b = [7, -7, 4] <br>
c = [2, -3, 3] <br>
Optimality: 28 <br>

##### Input 3: 3x3 matrix
A = [[1, 1, 3],[2, 2, 5],[4, 1, 2]] <br>
b = [30, 24, 36] <br>
c = [3, 1, 2] <br>
Optimality: 28 <br>

##### Input 4: 2x4
A = [[1, 1, 0, 1],[2, 1, 1, 0]] <br>
b = [8, 10] <br>
c = [1, 1, 0, 0] <br>
Optimality: 8 <br>

##### Input 5: 5x5
A = [[1, 0, 3, 0, 2], [3, 0, 0, 0, 3], [2, 1, 1, 4, 1], [0, 2, 1, 2, 1], [1, 3, 2, 1, 0]] <br>
b = [40, 19, 31, 25, 27] <br>
c = [4, 1, 2, 5, 3] <br>
Optimality: 56.667  <br>

##### Input 6: 7x7
A = [[2, 2, 2, 2, 2, 2, 2], [3, 3, 3, 3, 3, 3, 3], [1, 2, 3, 1, 2, 3, 1], [2, 3, 1, 2, 3, 1, 2], [4, 4, 4, 4, 4, 4, 4], [3, 2, 1, 2, 3, 2, 5], [5, 0, 0, 9, 9, 1, 1]] <br>
b = [40, 60, 30, 35, 65, 20, 50] <br>
c = [1, 3, 5, 7, 6, 4, 2] <br>
Optimality: 79.63  <br>

##### Input 7: 2x10
A = [[1,4],[0, 9],[3, 1],[1, 2],[4, 2],[3, 9],[7, 5],[8, 3],[1, 4],[2, 4]] <br>
b = [15, 30, 25, 9, 12, 20, 17, 10, 11, 18] <br>
c = [5, 4] <br>
Optimality: 10.635 <br>

##### Input 8: 10x2
A = [[9, 2, 1, 4, 1, 0, 0, 0, 2, 3], [1, 3, 4, 2, 0, 1, 2, 1, 0, 0]] <br>
b = [40, 50] <br>
c = [1, 2, 4, 3, 9, 8, 3, 1, 1, 4] <br>
Optimality: 760 <br>

##### Input 9: 10x10
A = [[], [], [], [], [], [], [], [], [], []] <br>
b = [] <br>
c = [9, 2, 4, 8, 1, 1, 2, 3, 5, 9] <br>
Optimality:  <br>

##### Input 10: 15x15
A = [[], [], [], [], []] <br>
b = [] <br>
c = [] <br>
Optimality:  <br>



### Randomly Generated Test Data 
The following code will be used to generate additional test data of various sizes: 


In [ ]:
    
for i in range(10): 
    m = random.randint(1, 5)
    n = random.randint(1, 5)
    A = np.random.uniform(low=1, high=5, size=(m, n))
    b = np.random.uniform(low=0, high=100, size=m)
    c = np.random.uniform(low=1, high=5, size=n) 

    test = linprog(c*-1, method='revised simplex', A_ub=A, b_ub=b)
    x, v = simplex(A, b, c)

    assert np.isclose(-test.fun, v), (-test.fun, 'does not equal', v)
    assert np.allclose(test.x, x)


## References
Carreira-Perpiñán, M. Á. (n.d.). Simplex Method. Retrieved from http://mathworld.wolfram.com/SimplexMethod.html
    
Cormen, T. H., Leiserson, C. E., Rivest, R. L., & Stein, C. (2009). Introduction to algorithms (Third Edition). Cambridge, MA: MIT Press.
